In [1]:
import numpy as np
import os, glob, sys, toml
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from itertools import combinations
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import Normalize
import matplotlib.cm as cm

import numpy as np
import copy
def fast_non_dominated_sort(Y):
    ''' a fast non-dominated sorting method
        Y: output objective matrix
    '''
    N, d = Y.shape
    Q = [] # temp array of Pareto front index
    Sp = [] # temp array of points dominated by p
    S = [] # temp array of Sp
    rank = np.zeros(N) # Pareto rank
    n = np.zeros(N)  # domination counter of p
    dom = np.zeros((N, N))  # the dominate matrix, 1: i doms j, 2: j doms i

    # compute the dominate relationship online, much faster
    for i in range(N):
        for j in range(N):
            if i != j:
                if dominates(Y[i,:], Y[j,:]):
                    dom[i,j] = 1
                    Sp.append(j)
                elif dominates(Y[j,:], Y[i,:]):
                    dom[i,j] = 2
                    n[i] += 1
        if n[i] == 0:
            rank[i] = 0
            Q.append(i)
        S.append(copy.deepcopy(Sp))
        Sp = []

    F = []
    F.append(copy.deepcopy(Q))
    k = 0
    while len(F[k]) > 0:
        Q = []
        for i in range(len(F[k])):
            p = F[k][i]
            for j in range(len(S[p])):
                q = S[p][j]
                n[q] -= 1
                if n[q] == 0:
                    rank[q]  = k + 1
                    Q.append(q)
        k += 1
        F.append(copy.deepcopy(Q))

    return rank, dom

def dominates(p,q):
    ''' comparison for multi-objective optimization
        d = True, if p dominates q
        d = False, if p not dominates q
        p and q are 1*nOutput array
    '''
    if sum(p > q) == 0:
        d = True
    else:
        d = False
    return d


In [10]:
# basin range (start from 1)
bstart = 1
bend = 300

# iteration range (start from 0)
istart = 0
iend = 1

path0 = f'/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/'

maxiter = 4 # used for plotting

In [11]:
# basin info
file = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv'
ds_info = pd.read_csv(file)
# ds_info.head()

In [13]:
# Create a PDF file to save the plots

# Create a colormap
# cmap = cm.tab20 
cmap = cm.cool
norm = Normalize(vmin=0, vmax=maxiter-1)  # Set the normalization from 0 to 15

values = np.linspace(0, maxiter-1, maxiter)
colors = cmap(norm(values))

minobjfuncs_all = []

file_plot = f'lse_camels_b{bstart}-{bend}_i{istart}-{iend}.pdf'
with PdfPages(file_plot) as pdf:

    for b in range(bstart-1, bend):
        fig = plt.figure(figsize=[8, 6])

        # load basin metrics
        df_met = pd.DataFrame()
        minobjfuncs = []
        emulator_metric = []
        for i in range(istart, iend+1):
            filemet = f'{path0}/level1_{b}_MOASMOcalib/ctsm_outputs/iter{i}_all_metric.csv'
            dfi = pd.read_csv(filemet)
            dfi['iter'] = i
            minobjfuncs.append( np.min(dfi['metric1'].values + dfi['metric2'].values) )
            df_met = pd.concat([df_met, dfi])

            # # load emulator metric
            # filemet1 = f'{path0}/level1_{b}_MOASMOcalib/param_sets/GPR_for_iter{i}_CV_kge.csv'
            # filemet2 = f'{path0}/level1_{b}_MOASMOcalib/param_sets/RF_for_iter{i}_CV_kge.csv'
            # df1 = pd.read_csv(filemet1)
            # df2 = pd.read_csv(filemet2)
            
            # emulator_metric.append(max(df1['kge_mean'].values[-1], df2['kge_mean'].values[-1]))
    
        met1 = df_met['metric1'].values
        met2 = df_met['metric2'].values
        iters = df_met['iter'].values

        # 1. plot all trials
        fig.add_subplot(2,2,1)
        
        plt.scatter(met1, met2, 5, iters, cmap=cmap, norm=norm)
        plt.colorbar(label='iteration')
        plt.xlabel('daily mean abs error')
        plt.ylabel('max monthly abs error')
        plt.title('(a) All interations')

        # plot the default parameter
        if istart == 0:
            plt.scatter(met1[0], met2[0], 5, color='r')

         # 2. plot all trials (two different colors for the current and previous iterations)
        fig.add_subplot(2,2,2)

        ind = iters==iters[-1]
        plt.scatter(met1[~ind], met2[~ind], 5, color='grey')
        plt.scatter(met1[ind], met2[ind], 5, color='blue')
        plt.xlabel('daily mean abs error')
        plt.ylabel('max monthly abs error')
        plt.title('(b) Current iter. VS Previous iter.')

        # plot the default parameter
        if istart == 0:
            plt.scatter(met1[0], met2[0], 10, color='r')
       

        # 3. plot the performance change (best performance for each iteration)
        fig.add_subplot(2,2,3)
        
        if istart == 0:
            minobjfuncs.insert(0, met1[0] + met2[0])
            minobjfuncs = np.array(minobjfuncs)
            iternames = list(np.arange(istart, iend+1))
            iternames.insert(0, 'defa')
        else:
            minobjfuncs = np.array(minobjfuncs)
            iternames = list(np.arange(istart, iend+1))

        minobjfuncs_all.append(minobjfuncs)
        
        plt.plot(np.arange(len(minobjfuncs)), minobjfuncs, 'o-', color='blue')
        plt.xticks(np.arange(len(minobjfuncs)),  iternames)
            
        plt.xlabel('iteration')
        plt.ylabel('min of metric 1 + metric 2')
        plt.title('(c) Best performance for each iteration')

        # 4. insert text
        fig.add_subplot(2,2,4)
        plt.axis('off')
        s = ds_info.iloc[b]['hru_id']
        plt.text(0.1, 0.8, f'hru_id: {s}')
        s = ds_info.iloc[b]['lat_cen']
        plt.text(0.1, 0.7, f'lat_cen: {s}')
        s = ds_info.iloc[b]['lon_cen']
        plt.text(0.1, 0.6, f'lon_cen: {s}')
        s = ds_info.iloc[b]['AREA']/1e6
        plt.text(0.1, 0.5, f'AREA: {s:.2f} km2')
        s = ds_info.iloc[b]['elev_mean']
        plt.text(0.1, 0.4, f'elev_mean: {s:.2f} m')

        plt.tight_layout()
        pdf.savefig(fig)  # Save the current figure into the PDF
        plt.close(fig)  # Close the figure to free memory
